In [ ]:
!pip install -r requirements.txt

In [ ]:
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Instantiate GitHub Toolkit
toolkit = GitHubToolkit.from_github_api_wrapper(GitHubAPIWrapper())
tools = [setattr(tool, "name", tool.mode) or tool for tool in toolkit.get_tools()]

llm = ChatOpenAI(model="gpt-4o", temperature=0)

agent_executor = create_react_agent(llm, tools)

def ai(query: str):
    # System prompt allows for broad range of instructions. But what does user input look like?
    events = agent_executor.stream(
        {"messages": [("system",
                       "You are a web server, serving HTML web pages. Do not use markdown for formatting. "
                       "All output must be in HTML format and will be displayed to an end user. "
                       "You need to start with doctype and html tags and provide the entire page"),
                      ("user", query)]},
        stream_mode="values",
    )
    data = ""
    for event in events:
        event["messages"][-1].pretty_print()
        data = event["messages"][-1]

    return data.content


In [ ]:
from pathlib import Path

from dotenv import load_dotenv
from flask import Flask, request


# Load environment variables from .env
load_dotenv()

app = Flask(__name__)

@app.route('/', methods=['GET'])
def get_data():
    # Wow where is the logic?
    return ai(Path("my_app-step-3.txt").read_text())


## Goals
- Write natural language only
- Unstructured data
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~

In [ ]:
app.run(debug=False, port=5001, use_reloader=False)